# Production-Ready Preprocessing

In [ ]:
testval_transforms = T.Compose(
    [
        # The size here depends on your application. Here let's use 256x256
        T.Resize(256),
        # Let's take the central 224x224 part of the image
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

We need to wrap our model in a wrapper class that is going to take care of applying the transformations and then run the transformed image through the CNN.

If we trained with the nn.CrossEntropyLoss as the loss function, we also need to apply a softmax function to the output of the model so that the output of the wrapper will be probabilities and not merely scores.

Let's see an example of such a wrapper class:

In [ ]:
import torch
from torchvision import datasets
import torchvision.transforms as T
from __future__ import annotations

class Predictor(nn.Module):

    def __init__(
            self,
            model: nn.Module,
            class_names: list[str],
            mean: torch.Tensor,
            std: torch.Tensor
    ):

        super().__init__()

        self.model = model.eval()
        self.class_names = class_names

        self.transforms = nn.Sequential(
            T.Resize([256, ]),
            T.CenterCrop(224),
            T.ConvertImageDtype(torch.float),
            T.Normalize(mean.tolist(), std.tolist())
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            # 1. apply transforms
            x = self.transforms(x)  # =
            # 2. get the logits
            x = self.model(x)  # =
            # 3. apply softmax
            #    HINT: remmeber to apply softmax across dim=1
            x = F.softmax(x, dim=1)  # =

            return x

# The Constructor

In [ ]:
self.transforms = nn.Sequential(
    T.Resize([256, ]),  # We use single int value inside a list due to torchscript type restrictions
    T.CenterCrop(224),
    T.ConvertImageDtype(torch.float),
    T.Normalize(mean.tolist(), std.tolist())
)

# The forward Method

In [ ]:
def forward(self, x: torch.Tensor) -> torch.Tensor:
    with torch.no_grad():
        # 1. apply transforms
        x = self.transforms(x)  # =
        # 2. get the logits
        x = self.model(x)  # =
        # 3. apply softmax
        #    HINT: remmeber to apply softmax across dim=1
        x = F.softmax(x, dim=1)  # =

        return x

# Export Using torchscript

In [ ]:
predictor = Predictor(model, class_names, mean, std).cpu()

# Export using torch.jit.script
scripted_predictor = torch.jit.script(predictor)
scripted_predictor.save("standalone_model.pt")

# All together the code will be:

In [ ]:
from PIL import Image
import torch
import torchvision
import torchvision.transforms as T

# Reload the model
learn_inf = torch.jit.load("standalone_model.pt")

# Read an image and transform it to tensor to simulate what would
# happen in production
img = Image.open("static_images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg")
# We use .unsqueeze because the model expects a batch, so this
# creates a batch of 1 element
pil_to_tensor = T.ToTensor()(img).unsqueeze_(0)

# Perform inference and get the softmax vector
softmax = predictor_reloaded(pil_to_tensor).squeeze()
# Get index of the winning label
max_idx = softmax.argmax()
# Print winning label using the class_names attribute of the 
# model wrapper
print(f"Prediction: {learn_inf.class_names[max_idx]}")